### Requirements

#### Python packages

The following packages are not required by <i>nscsim</i> but are needed to run this notebook

- [cython](https://pypi.org/project/Cython/)
- [MDAnalysis](https://pypi.org/project/MDAnalysis/)
- [periodictable](https://pypi.org/project/periodictable/)

### Requirements

In [30]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.ion()
import numpy as np
import MDAnalysis as mda
import periodictable
import nscsim
from nscsim.quasielastic import coherent as coh
from nscsim import plotting

### Download water simulation
`wget` and `tar` commands are necessary.  
You may skip this step if running the notebook more than once

In [ ]:
%%capture
%%bash
wget https://www.dropbox.com/s/7imfnz6ma1kh41w/water_4000.tar.gz
tar zxf water_4000.tar.gz

### Create system and extract coordinates

`u` is a `MDAnalysis.Universe` object. It contains 154 Lithium atoms, 154 Chloride atoms, and 1128 TIP3 water molecules.

In [31]:
u = mda.Universe('topology.psf', 'trajectory.dcd')

We investigate the trajectory.

In [32]:
print('number of conformations =', u.trajectory.n_frames)
print('Size of the box, in Angstroms =', u.dimensions[0:3])
print('It is a cubic box, as evidenced by the angles between box sides: ',u.dimensions[3:])

number of conformations = 4000
Size of the box, in Angstroms = [33.6035 33.6035 33.6035]
It is a cubic box, as evidenced by the angles between box sides:  [90. 90. 90.]


Select only the hydrogens and oxygens. There should be three atoms per water molecule. A look at file <i>topology.psf</i> shows that water hydrogens have type `HT` and water oxygens have type `OT`

In [33]:
sel = u.select_atoms('type HT or type OT')
print('number of atoms selected =', len(sel))
print('number of atoms per water molecule = ', len(sel)/len(sel.residues))

number of atoms selected = 3384
number of atoms per water molecule =  3.0


Obtain the bound coherent neutron scattering lengths for the atoms of our selection `sel`. Notice we have to "translate" from NAMD atom types to the name of the atomic element. We use dictionary `type_to_element` for this. Then we iterate over our selected atoms

In [34]:
type_to_element = dict(HT='H', OT='O')
#
# We use a for loop here
#
b_c = list()
for atom in sel:
    atomic_element = type_to_element[atom.type]
    coherent_length = getattr(periodictable, atomic_element).neutron.b_c
    b_c.append(coherent_length)
b_c = np.asarray(b_c)
#
# Advanced python: the previous lines can be substituted with the following one-liner
#
#b_c = np.asarray([getattr(periodictable, type_to_element[at.type]).neutron.b_c for at in sel])


### Calculate the coherent dynamic structure factor `S(q)`

We calculate scattering only for Q=0.5 using the first 1000 frames in the trajectory. The liquid allows to compute scattering only as a function of Q-vector modulus so we carry out a spherical average for all orientations of vector Q (function `intermediate_spherical`).

Notice we call `intermediate_spherical` only with one core (`n_cores=1`) because in most computers numpy will automatically use parallel versions of underlying computing BLAS libraries. Thus, all cores will be used even if we require one core. If this is not the case for your computer, then you can request more cores by changing the value of keyword `n_cores`.

In [ ]:
tr = np.asarray([sel.positions for _ in u.trajectory[0:1000:1]])  # extract coordinates to numpy array
q_mod = np.array([0.5, ])
sf = coh.intermediate_spherical(tr, q_mod, b_c, n_cores=1)
times = coh.times(tr)

Simple plotting of the curve.

In [ ]:
plotting.single_curve(times, sf[0], label='time', ylabel='Coherent Intensity')

Notice that at long times the scattering is not reliable due to low statistical sampling. A more reliable curve at times~1000 can be obtained if we use all frames, not just the first 1000:

<code>tr = np.asarray([sel.positions for _ in u.trajectory[::1]]) # all frames in the trajectory</code>

### Calculate the coherent dynamic structure factor `S(q)` in a log scale of Q-values

We will calculate more values of Q using the first 100 oxygen atoms and all the trajectory frames. Here we calculate scattering from `Q=0.01` to `Q=10` for a total of 15 Q-values spaced in a log scale.

In [ ]:
sel = u.select_atoms('type OT')[0:100]
tr = np.asarray([sel.positions for _ in u.trajectory[::1]])  # extract coordinates to numpy array
b_c = np.asarray([getattr(periodictable, 'O').neutron.b_c for at in sel])

q_mod = nscsim.qvec.moduli_logscale(min_exp=-2, max_exp=1, n_per_base=5, base=10)

In [ ]:
sf = coh.intermediate_spherical(tr, q_mod, b_c, n_cores=1)
times = coh.times(tr)

Plot all curves sequentially

In [ ]:
common = dict(xlabel='time', ylabel='Coherent Intensity', xscale='linear', yscale='linear')
for i, curve in enumerate(sf):
    common['label'] = 'Q={:.2f}'.format(q_mod[i])
    plotting.single_curve(times, curve, **common)